In [1]:
from analyze_csv import *
from mutations import get_all_moves
from evaluation_indices import *
from batch_tasks import *
from data_generation import *
from individual import Individual
import numpy as np
import pandas as pd
import os
import traceback
import pickle
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from scipy.stats import entropy, skew, kurtosis
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import f1_micro
from time import time
from random import shuffle

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
all_moves = np.array(get_all_moves())
all_moves

array(['unguided_merge_gene_move', 'unguided_remove_and_reclassify_move',
       'unguided_split_gene_move', 'expand_cluster_move',
       'split_farthest_move', 'unguided_eliminate_move',
       'knn_reclassification_move', 'one_nth_change_move',
       'centroid_k_means_move', 'centroid_hill_climbing_move',
       'prototype_k_means_move', 'prototype_hill_climbing_move',
       'tree_hill_climbing_move',
       'guided_remove_and_reclassify_move("density_based_validity_separation")',
       'guided_remove_and_reclassify_move("density_based_sparseness_separation")',
       'guided_remove_and_reclassify_move("mean_centroid_distance_separation")',
       'guided_remove_and_reclassify_move("diameter_separation")',
       'guided_histogram_split_gene_move("density_based_validity_separation")',
       'guided_histogram_split_gene_move("density_based_sparseness_separation")',
       'guided_histogram_split_gene_move("mean_centroid_distance_separation")',
       'guided_histogram_split_gene_

In [3]:
index_minimized = {}
for name, func in indices:
    index_minimized[name] = eval(func).is_minimized
index_minimized

{'silhouette': False,
 'calinski_harabaz': False,
 'davies_bouldin': True,
 'dvcb': False,
 'dunn': False,
 'generalized_dunn_41': False,
 'generalized_dunn_43': False,
 'generalized_dunn_51': False,
 'generalized_dunn_53': False,
 'generalized_dunn_13': False}

In [16]:
if True:
    mutation_usage = {}
    def parse_folder(path):
        for idx, filename in enumerate(os.listdir(path)):
            index_name = filename[:filename.find('-')]
            try:
                data, time = read_results(path + '/' + filename)
            except:
                traceback.print_exc()
                continue
            n_clusters = data.iloc[-1]['n_clusters']
            unique_numbers = np.unique(np.bincount(data.index[1:]))
            mutation_names = np.array([s[:s.find(':')] if s is not None and s != 'None' else s for s in data['detail']])
            mut_indices = np.argwhere(mutation_names[:, None] == all_moves[None, :])[:, 1]
            assert len(mut_indices) == len(data) - 1
            data['mutation_index'] = np.hstack([[-1], mut_indices])
            index_series = [data[data['mutation_index'] == i]['index'].rename(all_moves[i] + '_index') for i in range(len(all_moves))]            
            assert all(b == a + '_index' for a, b in zip(all_moves, (s.name for s in index_series)))
            table = pd.concat(index_series, axis=1).values
            if not index_minimized[index_name]:
                max_indices = np.nanargmax(table, axis=1)
                max_values = np.nanmax(table, axis=1)
                change_steps = np.argwhere(max_values[1:] > max_values[:-1]).flatten() + 1
            else:
                max_indices = np.nanargmin(table, axis=1)
                max_values = np.nanmin(table, axis=1)
                change_steps = np.argwhere(max_values[1:] < max_values[:-1]).flatten() + 1
            c_usage = np.unique(max_indices[change_steps])
            diff = 0 if filename not in mutation_usage else len(np.setdiff1d(c_usage, mutation_usage[filename])) + len(np.setdiff1d(mutation_usage[filename], c_usage))
            mutation_usage[filename] = c_usage if filename not in mutation_usage else np.unique(np.hstack([c_usage, mutation_usage[filename]]))
            print(idx, filename, n_clusters, len(unique_numbers), len(c_usage), len(change_steps), diff)
            
    parse_folder('/tmp/results-1010')
    parse_folder('/tmp/results2')
    mutation_usage = list(mutation_usage.items())
    with open('mutation_usage.dat', 'wb') as f:
        pickle.dump(mutation_usage, f)
else:
    with open('mutation_usage.dat', 'rb') as f:
        mutation_usage = pickle.load(f)

0 calinski_harabaz-Concrete_Data-one_plus_lambda_all_moves.txt 3 4 6 12 0
1 calinski_harabaz-HappinessRank_2015-one_plus_lambda_all_moves.txt 4 5 2 3 0
2 calinski_harabaz-ICU-one_plus_lambda_all_moves.txt 4 5 2 2 0
3 calinski_harabaz-ParkinsonSpeechDataset-one_plus_lambda_all_moves.txt 4 6 4 12 0
4 calinski_harabaz-analcatdata_lawsuit-one_plus_lambda_all_moves.txt 2 7 2 2 0
5 calinski_harabaz-analcatdata_vineyard-one_plus_lambda_all_moves.txt 2 7 3 7 0
6 calinski_harabaz-analcatdata_wildcat-one_plus_lambda_all_moves.txt 5 6 6 15 0
7 calinski_harabaz-arsenic_female_bladder-one_plus_lambda_all_moves.txt 2 6 5 6 0
8 calinski_harabaz-arsenic_male_bladder-one_plus_lambda_all_moves.txt 3 5 5 7 0
9 calinski_harabaz-arsenic_male_lung-one_plus_lambda_all_moves.txt 2 5 5 5 0
10 calinski_harabaz-autoUniv_au7_1100-one_plus_lambda_all_moves.txt 4 6 4 10 0
11 calinski_harabaz-autoUniv_au7_500-one_plus_lambda_all_moves.txt 4 5 3 3 0
12 calinski_harabaz-autoUniv_au7_700-one_plus_lambda_all_moves.txt 4

108 davies_bouldin-autoUniv_au7_500-one_plus_lambda_all_moves.txt 3 7 2 2 0
109 davies_bouldin-autoUniv_au7_700-one_plus_lambda_all_moves.txt 17 2 10 15 0
110 davies_bouldin-badges2-one_plus_lambda_all_moves.txt 12 2 6 9 0
111 davies_bouldin-banknote_authentication-one_plus_lambda_all_moves.txt 47 2 25 198 0
112 davies_bouldin-boston_corrected-one_plus_lambda_all_moves.txt 3 7 3 4 0
113 davies_bouldin-breast_tissue-one_plus_lambda_all_moves.txt 19 2 7 18 0
114 davies_bouldin-chscase_census2-one_plus_lambda_all_moves.txt 3 7 4 6 0
115 davies_bouldin-chscase_census6-one_plus_lambda_all_moves.txt 3 7 5 6 0
116 davies_bouldin-collins-one_plus_lambda_all_moves.txt 3 8 4 7 0
117 davies_bouldin-dataset_185_yeast-one_plus_lambda_all_moves.txt 7 11 13 18 0
118 davies_bouldin-dataset_187_abalone-one_plus_lambda_all_moves.txt 26 2 13 32 0
119 davies_bouldin-dataset_18_mfeat_morphological-one_plus_lambda_all_moves.txt 41 2 22 183 0
120 davies_bouldin-dataset_2181_auto_price-one_plus_lambda_all_mov

220 dunn-dataset_41_glass-one_plus_lambda_all_moves.txt 4 7 1 1 0
221 dunn-dataset_43_haberman-one_plus_lambda_all_moves.txt 4 7 1 1 0
222 dunn-dataset_53_heart_statlog-one_plus_lambda_all_moves.txt 4 3 2 2 0
223 dunn-dataset_54_vehicle-one_plus_lambda_all_moves.txt 4 5 2 2 0
224 dunn-dataset_8_liver_disorders-one_plus_lambda_all_moves.txt 2 5 0 0 0
225 dunn-debutanizer-one_plus_lambda_all_moves.txt 4 3 6 8 0
226 dunn-disclosure_x_bias-one_plus_lambda_all_moves.txt 8 2 1 1 0
227 dunn-fri_c0_1000_25-one_plus_lambda_all_moves.txt 3 7 3 4 0
228 dunn-fri_c0_100_25-one_plus_lambda_all_moves.txt 3 6 2 2 0
229 dunn-fri_c0_250_10-one_plus_lambda_all_moves.txt 3 6 2 2 0
230 dunn-fri_c1_1000_5-one_plus_lambda_all_moves.txt 3 7 1 4 0
231 dunn-fri_c1_100_10-one_plus_lambda_all_moves.txt 2 6 0 0 0
232 dunn-fri_c1_100_5-one_plus_lambda_all_moves.txt 3 6 0 0 0
233 dunn-fri_c1_250_10-one_plus_lambda_all_moves.txt 3 6 1 1 0
234 dunn-fri_c1_250_5-one_plus_lambda_all_moves.txt 3 7 2 2 0
235 dunn-fri_c1_5

347 dvcb-generated_10dim_10cl-one_plus_lambda_all_moves.txt 7 2 5 11 0
348 dvcb-generated_10dim_30cl-one_plus_lambda_all_moves.txt 18 2 14 18 0
349 dvcb-generated_2dim_10cl-one_plus_lambda_all_moves.txt 9 2 13 22 0
350 dvcb-generated_2dim_30cl-one_plus_lambda_all_moves.txt 7 2 7 24 0
351 dvcb-heart_h-one_plus_lambda_all_moves.txt 4 3 6 7 0
352 dvcb-heart_long_beach-one_plus_lambda_all_moves.txt 4 6 9 13 0
353 dvcb-heart_switzerland-one_plus_lambda_all_moves.txt 4 6 15 20 0
354 dvcb-immunotherapy-one_plus_lambda_all_moves.txt 2 8 8 12 0
355 dvcb-iris-one_plus_lambda_all_moves.txt 5 7 8 9 0
356 dvcb-leaf-one_plus_lambda_all_moves.txt 5 3 7 11 0
357 dvcb-mu284-one_plus_lambda_all_moves.txt 2 8 3 3 0
358 dvcb-oil_spill-one_plus_lambda_all_moves.txt 4 3 13 19 0
359 dvcb-parkinsons-one_plus_lambda_all_moves.txt 4 6 12 15 0
360 dvcb-pathogen_survey_dataset-one_plus_lambda_all_moves.txt 3 7 4 5 0
361 dvcb-planning_relax-one_plus_lambda_all_moves.txt 2 8 3 4 0
362 dvcb-pm10-one_plus_lambda_all_

457 generalized_dunn_13-pathogen_survey_dataset-one_plus_lambda_all_moves.txt 3 3 2 2 0
458 generalized_dunn_13-planning_relax-one_plus_lambda_all_moves.txt 3 8 1 2 0
459 generalized_dunn_13-pm10-one_plus_lambda_all_moves.txt 4 7 2 3 0
460 generalized_dunn_13-pollen-one_plus_lambda_all_moves.txt 3 8 6 8 0
461 generalized_dunn_13-prnn_crabs-one_plus_lambda_all_moves.txt 2 4 0 0 0
462 generalized_dunn_13-prnn_fglass-one_plus_lambda_all_moves.txt 4 7 1 1 0
463 generalized_dunn_13-quake-one_plus_lambda_all_moves.txt 4 5 4 6 0
464 generalized_dunn_13-rmftsa_ctoarrivals-one_plus_lambda_all_moves.txt 3 5 3 3 0
465 generalized_dunn_13-rmftsa_sleepdata-one_plus_lambda_all_moves.txt 2 7 3 3 0
466 generalized_dunn_13-sales_transactions-one_plus_lambda_all_moves.txt 144 3 24 117 0
467 generalized_dunn_13-seeds-one_plus_lambda_all_moves.txt 4 4 1 1 0
468 generalized_dunn_13-seismic_bumps-one_plus_lambda_all_moves.txt 2 7 1 1 0
469 generalized_dunn_13-steel_plates_fault-one_plus_lambda_all_moves.txt

...

469 generalized_dunn_13-steel_plates_fault-one_plus_lambda_all_moves.txt 3 7 4 6 6
470 generalized_dunn_13-transplant-one_plus_lambda_all_moves.txt 4 6 1 1 1
471 generalized_dunn_13-user_knowledge-one_plus_lambda_all_moves.txt 4 4 1 1 1
472 generalized_dunn_13-vertebra_column-one_plus_lambda_all_moves.txt 3 6 1 1 9
473 generalized_dunn_13-volcanoes_a2-one_plus_lambda_all_moves.txt 4 4 3 4 4
474 generalized_dunn_13-volcanoes_a3-one_plus_lambda_all_moves.txt 4 6 3 3 5
475 generalized_dunn_13-volcanoes_a4-one_plus_lambda_all_moves.txt 4 4 4 5 8
476 generalized_dunn_13-volcanoes_e1-one_plus_lambda_all_moves.txt 2 6 3 3 4
477 generalized_dunn_13-volcanoes_e2-one_plus_lambda_all_moves.txt 5 2 1 1 2
478 generalized_dunn_13-volcanoes_e3-one_plus_lambda_all_moves.txt 4 4 2 3 4
479 generalized_dunn_13-volcanoes_e4-one_plus_lambda_all_moves.txt 3 2 1 1 3
480 generalized_dunn_13-volcanoes_e5-one_plus_lambda_all_moves.txt 4 4 2 2 5
481 generalized_dunn_13-vowel-one_plus_lambda_all_moves.txt 21 7 9 

572 generalized_dunn_41-volcanoes_a4-one_plus_lambda_all_moves.txt 9 3 16 53 12
573 generalized_dunn_41-volcanoes_e1-one_plus_lambda_all_moves.txt 11 3 12 59 9
574 generalized_dunn_41-volcanoes_e2-one_plus_lambda_all_moves.txt 4 6 13 32 7
575 generalized_dunn_41-volcanoes_e3-one_plus_lambda_all_moves.txt 4 6 10 19 8
576 generalized_dunn_41-volcanoes_e4-one_plus_lambda_all_moves.txt 4 6 10 24 7
577 generalized_dunn_41-volcanoes_e5-one_plus_lambda_all_moves.txt 4 5 9 33 8
578 generalized_dunn_41-vowel-one_plus_lambda_all_moves.txt 3 7 9 12 7
579 generalized_dunn_41-wholesale_customers-one_plus_lambda_all_moves.txt 3 6 2 2 7
580 generalized_dunn_41-wine_quality_red-one_plus_lambda_all_moves.txt 2 11 8 10 10
581 generalized_dunn_41-yagoSchema_ttl-one_plus_lambda_all_moves.txt 2 7 1 1 1
582 generalized_dunn_43-Concrete_Data-one_plus_lambda_all_moves.txt 5 4 11 124 10
583 generalized_dunn_43-HappinessRank_2015-one_plus_lambda_all_moves.txt 3 7 1 1 15
584 generalized_dunn_43-ICU-one_plus_lamb

674 generalized_dunn_43-volcanoes_e5-one_plus_lambda_all_moves.txt 7 3 11 40 8
675 generalized_dunn_43-vowel-one_plus_lambda_all_moves.txt 18 3 24 152 3
676 generalized_dunn_43-wholesale_customers-one_plus_lambda_all_moves.txt 3 7 2 2 4
677 generalized_dunn_43-wine_quality_red-one_plus_lambda_all_moves.txt 3 8 6 7 9
678 generalized_dunn_43-yagoSchema_ttl-one_plus_lambda_all_moves.txt 2 6 1 1 3
679 generalized_dunn_51-Concrete_Data-one_plus_lambda_all_moves.txt 4 4 6 11 8
680 generalized_dunn_51-HappinessRank_2015-one_plus_lambda_all_moves.txt 3 6 1 1 4
681 generalized_dunn_51-ICU-one_plus_lambda_all_moves.txt 3 8 6 9 7
682 generalized_dunn_51-ParkinsonSpeechDataset-one_plus_lambda_all_moves.txt 3 7 4 5 5
683 generalized_dunn_51-analcatdata_lawsuit-one_plus_lambda_all_moves.txt 4 7 3 3 2
684 generalized_dunn_51-analcatdata_vineyard-one_plus_lambda_all_moves.txt 3 7 7 8 8
685 generalized_dunn_51-analcatdata_wildcat-one_plus_lambda_all_moves.txt 4 6 2 2 3
686 generalized_dunn_51-arsenic_f

777 generalized_dunn_53-HappinessRank_2015-one_plus_lambda_all_moves.txt 3 7 3 3 3
778 generalized_dunn_53-ICU-one_plus_lambda_all_moves.txt 4 5 4 6 4
779 generalized_dunn_53-ParkinsonSpeechDataset-one_plus_lambda_all_moves.txt 3 7 8 11 9
780 generalized_dunn_53-analcatdata_lawsuit-one_plus_lambda_all_moves.txt 4 8 11 17 7
781 generalized_dunn_53-analcatdata_vineyard-one_plus_lambda_all_moves.txt 2 7 8 19 3
782 generalized_dunn_53-analcatdata_wildcat-one_plus_lambda_all_moves.txt 3 6 1 2 3
783 generalized_dunn_53-arsenic_female_bladder-one_plus_lambda_all_moves.txt 3 6 11 15 10
784 generalized_dunn_53-arsenic_male_bladder-one_plus_lambda_all_moves.txt 4 8 9 11 6
785 generalized_dunn_53-arsenic_male_lung-one_plus_lambda_all_moves.txt 3 8 7 8 5
786 generalized_dunn_53-autoUniv_au7_1100-one_plus_lambda_all_moves.txt 3 7 10 22 9
787 generalized_dunn_53-autoUniv_au7_500-one_plus_lambda_all_moves.txt 4 5 10 30 2
788 generalized_dunn_53-autoUniv_au7_700-one_plus_lambda_all_moves.txt 4 10 14 9

880 silhouette-arsenic_female_bladder-one_plus_lambda_all_moves.txt 3 7 4 4 6
881 silhouette-arsenic_male_bladder-one_plus_lambda_all_moves.txt 4 7 4 5 9
882 silhouette-arsenic_male_lung-one_plus_lambda_all_moves.txt 3 7 1 1 3
883 silhouette-autoUniv_au7_1100-one_plus_lambda_all_moves.txt 5 3 6 17 11
884 silhouette-autoUniv_au7_500-one_plus_lambda_all_moves.txt 4 4 4 4 6
885 silhouette-autoUniv_au7_700-one_plus_lambda_all_moves.txt 4 5 4 7 6
886 silhouette-badges2-one_plus_lambda_all_moves.txt 3 7 3 3 3
887 silhouette-banknote_authentication-one_plus_lambda_all_moves.txt 4 6 15 35 6
888 silhouette-boston_corrected-one_plus_lambda_all_moves.txt 3 6 1 1 4
889 silhouette-breast_tissue-one_plus_lambda_all_moves.txt 4 4 2 5 1
890 silhouette-chscase_census2-one_plus_lambda_all_moves.txt 3 6 2 2 5
891 silhouette-chscase_census6-one_plus_lambda_all_moves.txt 3 7 2 3 1
892 silhouette-collins-one_plus_lambda_all_moves.txt 3 6 4 4 5
893 silhouette-dataset_185_yeast-one_plus_lambda_all_moves.txt 4

In [18]:
np.bincount(np.hstack([a[1] for  a in mutation_usage]))

array([436, 509, 248, 418, 234, 450, 264, 334, 447, 154, 281,  35, 329,
       473, 420, 350, 420, 196, 171, 145, 174, 247, 235, 198, 216, 384,
       283, 211, 267, 427, 408])

In [19]:
generated_prefix = 'scratch'
real_prefix = 'datasets'
tasks_list = init_batch(real_prefix)
datasDict = {a: {'dataname': a, 'dataset': eval(b)} for a, b in datas}
datasDict.keys()

Excluding column number 2 because it is a single value
Excluding column number 23 because it is a single value


dict_keys(['generated_2dim_10cl', 'generated_2dim_30cl', 'generated_10dim_10cl', 'generated_10dim_30cl', 'immunotherapy', 'user_knowledge', 'sales_transactions', 'Concrete_Data', 'HappinessRank_2015', 'ICU', 'ParkinsonSpeechDataset', 'analcatdata_lawsuit', 'analcatdata_vineyard', 'analcatdata_wildcat', 'arsenic_female_bladder', 'arsenic_male_bladder', 'arsenic_male_lung', 'autoUniv_au7_1100', 'autoUniv_au7_500', 'autoUniv_au7_700', 'badges2', 'banknote_authentication', 'boston_corrected', 'breast_tissue', 'chscase_census2', 'chscase_census6', 'collins', 'dataset_185_yeast', 'dataset_187_abalone', 'dataset_18_mfeat_morphological', 'dataset_2181_auto_price', 'dataset_36_segment', 'dataset_39_ecoli', 'dataset_41_glass', 'dataset_43_haberman', 'dataset_53_heart_statlog', 'dataset_54_vehicle', 'dataset_8_liver_disorders', 'debutanizer', 'disclosure_x_bias', 'fri_c0_1000_25', 'fri_c0_100_25', 'fri_c0_250_10', 'fri_c1_1000_5', 'fri_c1_100_10', 'fri_c1_100_5', 'fri_c1_250_10', 'fri_c1_250_5', 

In [20]:
for k, v in datasDict.items():
    dataset = v['dataset']
    v['ma1'] = np.log2(len(dataset))
    v['ma2'] = np.log2(dataset.shape[1])
    meta_attrs = []
    meta_entropy = []
    cont_attrs = []
    for i, s in enumerate(dataset.T):
        uniq = np.unique(s)
        uniq.sort()
        if len(uniq) < len(s) * 0.3:
            meta_attrs.append(i)
            ints = np.searchsorted(uniq, s)
            meta_entropy.append(entropy(ints))
        else:
            cont_attrs.append(i)
    v['ma3'] = len(meta_attrs)
    v['ma5'] = sum(meta_entropy) / len(meta_entropy) if len(meta_entropy) > 0 else 0
    # TODO ma6
    if len(cont_attrs) > 0:
        corr = np.abs(np.corrcoef(dataset[:, cont_attrs], rowvar=False))
        v['ma7'] = corr[np.arange(0, len(corr))[None, :] <= np.arange(0, len(corr))[:, None]].mean() if isinstance(corr, np.ndarray) else corr
        v['ma8'] = np.mean(skew(dataset[:, cont_attrs]))
        v['ma9'] = np.mean(kurtosis(dataset[:, cont_attrs]))
    else:
        v['ma7'], v['ma8'], v['ma9'] = 0, 0, 0
    dists = pdist(dataset)
    v['md1'] = dists.mean()
    v['md2'] = dists.var()
    v['md3'] = dists.std()
    v['md4'] = skew(dists)
    v['md5'] = kurtosis(dists)
    hist = np.histogram(dists, bins=9, range=(0, dists.max()))[0]
    assert len(hist) == 9
    for i, h in enumerate(hist):
        v['md' + str(i + 6)] = h
    z = np.abs(((dists - v['md1']) / v['md3']))
    v['md16'] = (z < 1).sum()
    v['md17'] = ((1 <= z) & (z < 2)).sum()
    v['md18'] = ((2 <= z) & (z < 3)).sum()
    v['md19'] = (3 <= z).sum()

In [21]:
from sklearn.cluster import KMeans, AffinityPropagation, AgglomerativeClustering, MeanShift
state_of_the_art = ['KMeans(n_clusters=n_clusters)', 'AffinityPropagation()', 'AgglomerativeClustering(n_clusters=n_clusters)', 'MeanShift()']

In [22]:
for k, v in datasDict.items():
    start = time()
    dataset = v['dataset']
    n_clusters = int(np.cbrt(len(dataset)))
    values = []
    fnum = 1
    for algo in state_of_the_art:
        try:
            cls = eval(algo).fit_predict(dataset)
        except:
            traceback.print_exc()
            cls = None
        indiv = Individual({'data': dataset, 'labels': cls})
        for _, idxstr in indices:
            try:
                idx = eval(idxstr)(indiv)
            except:
                if cls is not None:
                    traceback.print_exc()
                idx = np.nan
            v['ml' + str(fnum)] = idx
            values.append(idx)
            fnum += 1
    print(k, 'took', time() - start, 'seconds,', values)

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)


generated_2dim_10cl took 15.756421566009521 seconds, [nan, 20689.835549467487, 3.513413420143542, 0.04383234864563659, 0.037868448519518635, 0.5359873194599644, 0.12212828007784197, 4.113810919510055, 0.9373592134071678, 0.008628578175253945, -0.4792510995092671, 8.588687083807738, 14.709404485758322, -0.353384979420912, 0.0005227863271917354, 0.001173604068020754, 7.083524863059628e-05, 0.0017604061020311309, 0.00010625287294589443, 3.1553826777164825e-05, 0.7080526112138442, 20532.027486792307, 3.5235245084209232, 0.04490259890482793, 0.08152223388414177, 0.5133660881799031, 0.1169738818874883, 4.523546782766988, 1.0307202584337505, 0.018575383877374514, 0.3814387801551384, 4258.397449231275, 9.372024621639703, 0.273267171822439, 0.5122049534716551, 1.3299988859013268, 0.09272459668541633, 34.57504419086646, 2.4104960251947087, 0.03570980264298902]


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)
Traceback (most recent call last):
  File "<ipython-input-22-65b6e005b459>", line 16, in <module>
    idx = eval(idxstr)(indiv)
  File "/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/evaluation_indices.py", line 11, in __call__
    return idx(*args, **kwargs)
  File "/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/evaluation_indices.py", line 20, in silhouette_index
    return silhouette_score(indiv["data"], ind

generated_2dim_30cl took 18.319862365722656 seconds, [0.6531285210080675, 2252.2723227073557, 5.109952215573632, 0.11745088069411688, 0.056676282684684026, 0.7220312012425686, 0.10949359163988173, 5.201531086265349, 0.7887946112045534, 0.008594766737592094, nan, 2.5898242643660145, 71.20646890012677, 0.3010999859973719, 0.0001522438640055314, 0.0025739161807757155, 0.0009623365286879213, 0.0038608742711635736, 0.0014435047930318819, 5.6920980059639927e-05, 0.6424070111975099, 2219.320042355842, 5.459408548171235, 0.15134879749062186, 0.19607973875088028, 0.6763873234364891, 0.11773683768501302, 7.448785157122927, 1.2965890675463905, 0.03413104825404056, nan, nan, 0.0, -0.5090521831735889, nan, nan, nan, nan, nan, nan]


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)


...

/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)


wholesale_customers took 3.555304765701294 seconds, [0.37733635360906415, 152.85652248016822, 5.228420770172078, -0.3826576956661997, 0.034822002336402855, 0.21417557760161723, 0.10230719673534809, 1.547118801777431, 0.7390263138256361, 0.016633742668716943, -0.0059102194691670815, 23.2932751988697, 12.173504186223369, -0.4730244431486473, 0.004626987934453051, 0.055909829697287104, 0.017199757713546586, 0.39446305095633305, 0.12135019799075723, 0.001423418240530921, 0.3854413491724139, 141.68062854437574, 4.948000246649376, -0.3457744654920107, 0.05309368405841414, 0.2432692246591342, 0.09666134616071585, 1.6510116749834123, 0.6560180855369974, 0.021096408643177512, 0.4215610112595193, 39.15060518860407, 3.5027402974655004, -0.27292924126133233, 0.08907906159204873, 0.45924092999783894, 0.05925816001852593, 0.6888613949967584, 0.0888872400277889, 0.011494317995886365]


/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)


wine_quality_red took 29.497429609298706 seconds, [0.08824230984681203, 283.02369964036774, 8.96802218659106, -0.3188708567176932, 0.029444263522339798, 0.11282851745926938, 0.054112803795326965, 1.8419937051821857, 0.8834242105213639, 0.01412153319711442, 0.12334174010475162, 104.66281643684982, 4.082341940025896, -0.16275146577939126, 0.039611143237629085, 0.09013097547241108, 0.04259324876018356, 0.6636264832635006, 0.3136103624457082, 0.01871906155184171, 0.050393786922151636, 239.09586388495245, 11.696447559092611, -0.30321581360762717, 0.041189813324156145, 0.10718898454865108, 0.041480748510993586, 1.7128944587097346, 0.6628679669538593, 0.01593992419005144, 0.41147652599371126, 53.90640877208247, 7.432429951391653, -0.307638772796813, 0.05558641654022965, 0.43824706636361105, 0.037768663480979195, 0.7448319731127678, 0.06419052242788886, 0.004790504766732629]
yagoSchema_ttl took 0.9111425876617432 seconds, [0.3478866414440017, 200.00355284870724, 3.9296301307838064, -0.44529453

/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)


In [23]:
datanames = list(datasDict)
shuffle(datanames)
cv10 = [datanames[len(datanames)*i//10:len(datanames)*(i+1)//10] for i in range(10)]
dataset_to_cv = {}
for i, c in enumerate(cv10):
    for v in c:
        dataset_to_cv[v] = i
with open('cv10.dat', 'wb') as f:
    pickle.dump(cv10, f)

In [24]:
meta_knowledge = [{} for i in range(10)]
def get_meta_list(dataset, name):
    meta_knowledge_list = meta_knowledge[dataset_to_cv[dataset]]
    if name not in meta_knowledge_list:
        meta_knowledge_list[name] = []
    return meta_knowledge_list[name]

In [25]:
for fname, mutations in mutation_usage:
    index, data = fname.split('-')[:2]
    get_meta_list(data, 'index').append(index)
    metadata = datasDict[data]
    for k, v in metadata.items():
        if k == 'dataset': continue
        get_meta_list(data, k).append(v)
    arr = np.zeros(len(all_moves), dtype='bool')
    arr[mutations] = True
    for i, v in enumerate(arr):
        get_meta_list(data, 'rm' + str(i)).append(v)
for d in meta_knowledge:
    assert len(set(map(len, d.values()))) == 1

In [26]:
metaframes = [pd.DataFrame(d) for d in meta_knowledge]
metaframes[0].head()

,index,dataname,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,calinski_harabaz,autoUniv_au7_1100,10.103288,3.700440,10,6.398238,0.504410,0.191808,-1.522806,0.088893,...,False,False,False,False,True,False,True,False,True,False
1,calinski_harabaz,badges2,7.022368,3.459432,10,4.116018,1.000000,1.246867,1.991627,0.222216,...,True,True,False,False,False,False,False,False,False,False
2,calinski_harabaz,fri_c1_250_10,7.965784,3.459432,0,0.000000,0.263500,-0.038811,-1.128003,0.290846,...,False,False,False,False,False,False,True,True,False,False
3,calinski_harabaz,generated_2dim_30cl,9.874981,1.000000,0,0.000000,0.743656,-0.149542,-0.950350,0.026408,...,True,True,True,True,True,False,False,False,False,False
4,calinski_harabaz,heart_h,8.194757,3.807355,13,4.836237,1.000000,-0.455030,2.491825,0.157464,...,False,False,False,False,False,False,False,False,False,False


In [27]:
train_columns = [c for c in metaframes[0] if not c.startswith('rm') and c != 'dataname']
test_columns = [c for c in metaframes[0] if c.startswith('rm')]

In [28]:
indices_list = np.array(list(index_minimized))
for d in metaframes:
    d['index'] = np.argwhere(d['index'].values[:, None] == indices_list[None, :])[:, 1].flatten()
    d[test_columns] = d[test_columns].astype('int')
metaframes[0].head()

,index,dataname,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,autoUniv_au7_1100,10.103288,3.700440,10,6.398238,0.504410,0.191808,-1.522806,0.088893,...,0,0,0,0,1,0,1,0,1,0
1,1,badges2,7.022368,3.459432,10,4.116018,1.000000,1.246867,1.991627,0.222216,...,1,1,0,0,0,0,0,0,0,0
2,1,fri_c1_250_10,7.965784,3.459432,0,0.000000,0.263500,-0.038811,-1.128003,0.290846,...,0,0,0,0,0,0,1,1,0,0
3,1,generated_2dim_30cl,9.874981,1.000000,0,0.000000,0.743656,-0.149542,-0.950350,0.026408,...,1,1,1,1,1,0,0,0,0,0
4,1,heart_h,8.194757,3.807355,13,4.836237,1.000000,-0.455030,2.491825,0.157464,...,0,0,0,0,0,0,0,0,0,0


In [29]:
indices_list

array(['silhouette', 'calinski_harabaz', 'davies_bouldin', 'dvcb', 'dunn',
       'generalized_dunn_41', 'generalized_dunn_43',
       'generalized_dunn_51', 'generalized_dunn_53',
       'generalized_dunn_13'], dtype='<U19')

In [30]:
for i, d in enumerate(metaframes):
    d.to_csv('metaframe_{:02d}.csv'.format(i))

In [2]:
metaframes = [pd.read_csv('metaframe_{:02d}.csv'.format(i), index_col=0) for i in range(10)]
metaframes[0].head()

,index,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,md2,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,7.643856,4.321928,17,3.287976,0.522758,0.023865,0.174592,0.142564,0.002888,...,0,0,0,0,0,1,0,0,0,0
1,1,10.396605,2.321928,1,6.413459,0.655107,-0.122684,0.134221,0.066743,0.000965,...,0,0,0,0,0,1,1,1,0,0
2,1,8.413628,2.807355,7,5.592515,0.000000,0.000000,0.000000,0.081610,0.002688,...,0,0,0,0,1,0,0,0,1,0
3,1,9.370687,2.000000,0,0.000000,0.439122,0.349144,0.757310,0.030512,0.000141,...,0,0,0,0,1,1,0,1,0,0
4,1,7.965784,3.459432,0,0.000000,0.263500,-0.038811,-1.128003,0.290846,0.003762,...,0,0,0,0,1,1,1,0,0,0


In [17]:
metaframe_train = pd.concat(metaframes[:-1])
metaframe_train.head()

,index,dataname,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,arsenic_male_bladder,9.126704,2.321928,4,5.578625,1.000000,9.600139,104.073434,0.060484,...,0,1,0,1,1,0,1,0,0,0
1,1,breast_tissue,6.714246,3.321928,1,3.953497,0.293765,0.015479,-1.196633,0.225433,...,0,0,0,0,0,1,0,0,0,0
2,1,fri_c3_1000_10,9.965784,3.459432,1,6.086775,0.292593,0.228655,-0.705256,0.140777,...,0,0,0,1,1,0,1,0,0,0
3,1,fri_c4_500_25,8.965784,4.700440,0,0.000000,0.126482,0.107719,-0.941671,0.319531,...,0,0,0,0,1,1,1,0,0,0
4,1,heart_h,8.194757,3.807355,13,4.836237,1.000000,-0.455030,2.491825,0.157464,...,0,0,0,0,0,0,0,0,0,0


In [7]:
cls = AutoSklearnClassifier(n_jobs=4)
cls.fit(metaframe_train[train_columns], metaframe_train[test_columns], feat_type=['Categorical'] + ['Numerical'] * (len(train_columns) - 1), metric=f1_micro)

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

[WARNING] [2019-05-22 12:20:33,315:EnsembleBuilder(1791095845):39af627e4c2f33e5c297eec0322fbf1c] No models better than random - using Dummy Score!
[WARNING] [2019-05-22 12:20:33,363:EnsembleBuilder(1791095845):39af627e4c2f33e5c297eec0322fbf1c] No models better than random - using Dummy Score!


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv

[WARNING] [2019-05-22 12:41:27,157:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-05-22 12:41:27,157:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


...

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

[WARNING] [2019-05-22 13:10:17,259:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-05-22 13:10:17,259:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_f

AutoSklearnClassifier(delete_output_folder_after_terminate=False,
           delete_tmp_folder_after_terminate=False,
           disable_evaluator_output=False, ensemble_memory_limit=1024,
           ensemble_nbest=50, ensemble_size=50, exclude_estimators=None,
           exclude_preprocessors=None, get_smac_object_callback=None,
           include_estimators=None, include_preprocessors=None,
           initial_configurations_via_metalearning=25, logging_config=None,
           metadata_directory=None, ml_memory_limit=3072, n_jobs=4,
           output_folder=None, per_run_time_limit=360,
           resampling_strategy='holdout',
           resampling_strategy_arguments=None, seed=1, shared_mode=False,
           smac_scenario_args=None, time_left_for_this_task=3600,
           tmp_folder=None)

In [9]:
predicted = cls.predict(metaframes[-1][train_columns], n_jobs=4)

In [15]:
confusion_matrix(metaframes[-1][test_columns].values.flatten(), predicted.flatten())

array([[1924,  280],
       [ 390,  506]])

In [44]:
num_answers = len(test_columns) * len(metaframes[-1])
random_sample = np.zeros(num_answers, dtype='int')
random_sample[np.random.choice(num_answers, replace=False, size=metaframes[-1][test_columns].values.sum())] = 1
random_sample = random_sample.reshape(metaframes[-1][test_columns].values.shape)
f1_micro(metaframes[-1][test_columns], random_sample)

0.2857142857142857

In [31]:
metaframes_numbered = []
c_index = 0
for frame in metaframes:
    metaframes_numbered.append(frame.set_index(np.arange(c_index, c_index + len(frame))))
    c_index += len(frame)
metaframe = pd.concat(metaframes_numbered)
metaframe

,index,dataname,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,autoUniv_au7_1100,10.103288,3.700440,10,6.398238,0.504410,0.191808,-1.522806,0.088893,...,0,0,0,0,1,0,1,0,1,0
1,1,badges2,7.022368,3.459432,10,4.116018,1.000000,1.246867,1.991627,0.222216,...,1,1,0,0,0,0,0,0,0,0
2,1,fri_c1_250_10,7.965784,3.459432,0,0.000000,0.263500,-0.038811,-1.128003,0.290846,...,0,0,0,0,0,0,1,1,0,0
3,1,generated_2dim_30cl,9.874981,1.000000,0,0.000000,0.743656,-0.149542,-0.950350,0.026408,...,1,1,1,1,1,0,0,0,0,0
4,1,heart_h,8.194757,3.807355,13,4.836237,1.000000,-0.455030,2.491825,0.157464,...,0,0,0,0,0,0,0,0,0,0
5,1,immunotherapy,6.491853,3.000000,5,4.056313,0.550095,1.198514,3.989046,0.209660,...,0,0,0,0,1,0,0,0,0,0
6,1,leaf,8.409391,4.000000,3,5.642384,0.459966,0.949782,2.858173,0.151443,...,0,0,0,0,0,0,0,0,0,0
7,1,user_knowledge,8.011227,2.584963,1,5.194584,0.414597,0.314493,-0.787879,0.105067,...,0,0,0,0,1,0,0,0,1,0
8,1,volcanoes_a2,10.664447,2.000000,1,4.923619,0.524263,0.542907,-0.009363,0.028298,...,0,0,0,0,0,1,0,1,1,0
9,2,autoUniv_au7_1100,10.103288,3.700440,10,6.398238,0.504410,0.191808,-1.522806,0.088893,...,1,1,1,1,1,1,1,0,1,1


In [32]:
metaframe.to_csv('metaframe.csv')

In [52]:
datanames

NameError: name 'datanames' is not defined